# Vehicle Classification

## Evaluate the model


In [1]:
import os

from sklearn.metrics import accuracy_score, classification_report

from utils import utils
from models import resnet_50


## Experiment settings

In [2]:
CONFIG_YML = "../experiments/exp_004/config.yml"
WEIGHTS = "../experiments/exp_004/model.45-1.3484.h5"

In [3]:
config = utils.load_config(CONFIG_YML)
config

{'seed': 123,
 'data': {'directory': '/home/app/src/data/car_ims_v2/train',
  'labels': 'inferred',
  'label_mode': 'categorical',
  'validation_split': 0.2,
  'image_size': [224, 224],
  'batch_size': 32},
 'model': {'weights': 'imagenet',
  'input_shape': [224, 224, 3],
  'classes': 196,
  'dropout_rate': 0.35,
  'data_aug_layer': {'random_flip': {'mode': 'horizontal'},
   'random_rotation': {'factor': 0.1},
   'random_zoom': {'height_factor': 0.2, 'width_factor': 0.2},
   'random_contrast': {'factor': [0.2, 0.5]}}},
 'compile': {'optimizer': {'sgd': {'learning_rate': 0.0005, 'momentum': 0.98}},
  'loss': 'categorical_crossentropy',
  'metrics': ['accuracy']},
 'fit': {'epochs': 50,
  'callbacks': {'model_checkpoint': {'filepath': '/home/app/src/experiments/exp_004/model.{epoch:02d}-{val_loss:.4f}.h5',
    'save_best_only': True},
   'tensor_board': {'log_dir': '/home/app/src/experiments/exp_004/logs'},
   'early_stopping': {'monitor': 'val_loss',
    'patience': 3,
    'mode': 'min'

In [4]:
config['model']['classes']

196

In [5]:
MODEL_CLASSES = utils.get_class_names(config)

if len(MODEL_CLASSES) != config['model']['classes']:
    raise ValueError(
        "Number of classes doesn't match between your model "
        "and your data!"
    )

_dirname, _ = os.path.split(config['data']['directory'])
TEST_FOLDER = os.path.join(_dirname, 'test')

if not os.path.exists(TEST_FOLDER):
    raise ValueError("'test' folder not found!")
    
if len(os.listdir(TEST_FOLDER)) != config['model']['classes']:
    raise ValueError(
        "Number of classes doesn't match between your model "
        "and your testing dataset!"
    )
    
if set(os.listdir(TEST_FOLDER)) != set(MODEL_CLASSES):
    raise ValueError(
        "The name of the subfolders inside your test set "
        "doesn't match with the model classes!"
    )

## Load your model

In [7]:
cnn_model = resnet_50.create_model(weights=WEIGHTS)

print(cnn_model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                             

## Get predictions from testing dataset

In [11]:
predictions, labels = utils.predict_from_folder(
    folder=TEST_FOLDER, 
    model=cnn_model, 
    input_size=config["data"]["image_size"], 
    class_names=MODEL_CLASSES,
)

if len(predictions) != len(labels):
    raise ValueError(
        "The lenght of predictions and labels lists doesn't match!"
    )

if not isinstance(predictions[0], str):
    raise ValueError(
        "Model predictions should be represented as string. E.g: 'Acura RL Sedan 2012'"
    )

if not isinstance(labels[0], str):
    raise ValueError(
        "Ground true labels should be represented as string. E.g: 'Acura RL Sedan 2012'"
    )


In [12]:
print(classification_report(y_true=labels, y_pred=predictions))


                                                        precision    recall  f1-score   support

                            AM General Hummer SUV 2000       0.70      0.91      0.79        44
                             Acura Integra Type R 2001       0.88      0.80      0.83        44
                                   Acura RL Sedan 2012       0.50      0.56      0.53        32
                                   Acura TL Sedan 2012       0.83      0.91      0.87        43
                                  Acura TL Type-S 2008       0.75      0.95      0.84        42
                                  Acura TSX Sedan 2012       1.00      0.30      0.46        40
                              Acura ZDX Hatchback 2012       0.78      0.82      0.80        39
              Aston Martin V8 Vantage Convertible 2012       0.54      0.56      0.55        45
                    Aston Martin V8 Vantage Coupe 2012       0.80      0.49      0.61        41
                  Aston Martin Virage C

In [13]:
acc = accuracy_score(y_true=labels, y_pred=predictions)

print(f"Your model accuracy is {acc:.4f}!")

if acc < .3:
    raise ValueError("Your model accuracy is too low :(\nYou can do it better! :)")


Your model accuracy is 0.7480!
